In [1]:
from optimizer import *

### let's open the file 23 stocks' daily returns from January 4th, 2016 to December 28th, 2017 from Yahoo Finance. 

In [2]:
stock_returns = pd.read_excel("../FERM_3_Assignment_1.xlsx", sheet_name="Daily_Returns_Q1_Q2")

In [3]:
stock_returns.head()

,date,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
0,2016-01-04,-0.000736,0.000976,0.014852,-0.018058,0.000000,0.022343,0.008006,-0.023029,-0.010178,...,0.006963,0.026925,0.013954,0.002899,-0.032335,-0.004554,0.020011,0.019501,-0.000378,0.001867
1,2016-01-05,-0.005018,-0.016069,-0.006321,-0.045479,-0.021533,-0.000951,-0.008006,0.018804,-0.005128,...,-0.015441,-0.062643,-0.011614,-0.012273,-0.056744,-0.010708,0.010534,-0.017713,-0.019281,-0.015985
2,2016-01-06,-0.017237,-0.043235,-0.026306,-0.031773,-0.036736,-0.023581,-0.031023,0.074229,-0.012937,...,-0.016280,-0.049700,-0.040530,-0.014546,0.009889,-0.023338,-0.017561,-0.028090,-0.028942,-0.030801
3,2016-01-07,-0.009301,-0.018113,-0.001967,-0.012678,-0.019545,-0.006596,0.000000,-0.018247,-0.007843,...,0.000895,0.037504,0.016888,-0.006605,0.011009,-0.025106,0.001719,-0.025754,-0.016807,-0.017752
4,2016-01-08,0.012082,0.004559,-0.012135,0.018175,0.007211,0.011696,-0.001660,-0.055982,0.002621,...,0.012150,-0.035384,-0.016888,0.014431,0.012092,0.019581,0.020731,-0.010268,0.010637,0.004963


In [4]:
# compute mean and covariance matrix of these stocks

In [5]:
mean_return_matrix = stock_returns.drop(columns=["date"]).mean()
cov_matrix = stock_returns.drop(columns=["date"]).cov()

In [6]:
targeted_return1 = 0.0005

In [12]:
w_port, var_port, return_port, result_port = mean_variance_opt(mean_return_matrix, cov_matrix, targeted_return1)
print("portfolio weights:", w_port)
print("portfolio mean return : ", round(return_port, 4)*100, "%")
print("vol:", round(np.sqrt(var_port)*100, 4), "%")
print("variance:", round(var_port, 6))

portfolio weights: [ 0.08917668 -0.0208372   0.02284051 -0.04303578  0.05731254  0.02201725
 -0.02518617  0.07055633  0.06539291  0.01996536 -0.02433193  0.1721223
 -0.03507993  0.14724748  0.00426806  0.02246318  0.03250857  0.03563451
  0.06099552  0.23302199  0.04377781  0.03257602  0.016594  ]
portfolio mean return :  0.05 %
vol: 0.5765 %
variance: 3.3e-05
